In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

class KNN:
    def __init__(self, n, k=3, p=2, weights = 'uniform'):
        self.k = k
        self.p = p
        self.weights = weights
        self.n = n

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        if self.p==1:
            distances = [np.sum(np.abs(x - x_train)) for x_train in self.X_train]
        elif self.p==2:
            distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        k_neighbors_indices = np.argsort(distances)[:self.k]
        if self.weights == 'uniform':
            k_neighbor_labels = [self.y_train[i] for i in k_neighbors_indices]
            most_common = np.bincount(k_neighbor_labels).argmax()
        elif self.weights == 'distance':
            inverse_distances = np.array([1 / d if d!=0 else float('inf') for d in distances])[k_neighbors_indices]
            sum_of_inverse_distances = np.sum(inverse_distances)
            inverse_fraction = inverse_distances / sum_of_inverse_distances
            labels = self.y_train[k_neighbors_indices]
            sum=[0]*self.n
            for i in range(len(inverse_fraction)):
                sum[labels[i]] += inverse_fraction[i]
            most_common = sum.index(max(sum))
        return most_common

def find_k(n, X_train, y_train, X_test, y_test,  k_values, test_size=0.2):
    best_k = None
    best_p = None
    best_weights = None
    best_accuracy = 0.0
    p_values = [1,2]
    weight = ['uniform', 'distance']
    for k in k_values:
        for p in p_values:
            for weights in weight:
                knn = KNN(n, k=k, p=p, weights = weights)
                knn.fit(X_train, y_train)
                y_predicted = []
                y_pred = knn.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)*100
                print(f'Accuracy for k={k} with p{p} and weight {weights}: {accuracy}')
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_k = k
                    best_p = p
                    best_weights = weights
    return (best_k, best_p, best_weights)

if __name__ == "__main__":
    df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/Mobile.csv')
    X = df.drop('price_range',axis=1)
    y = df['price_range']
    df.drop('price_range',axis=1, inplace= True)
    X = np.array(X)
    y = np.array(y)
    n = len(set(y))
    k_values = range(1,11)  # Choose a range of k values to test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 35, shuffle= True, stratify = y)
    best_k, best_p, best_weights = find_k(n, X_train, y_train, X_test, y_test,  k_values)
    
    print(f"\nBest k value: {best_k}")
    print(f"\nBest p value: {best_p}")
    print(f"\nBest weights values: {best_weights}")

    # Create the final model with the best k
    final_model = KNN(n, k= best_k, p= best_p, weights= best_weights)
    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy of the Model: ", accuracy*100)
    from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)
    precision = precision_score(y_test, y_pred, average ='macro')
    print(precision)
    recall = recall_score(y_test, y_pred, average = 'macro')
    print(recall)
    f1score = f1_score(y_test, y_pred, average = 'macro')
    print(f1score)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,  recall_score, f1_score

df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/Mobile.csv')
X = df.drop('price_range',axis=1)
y = df['price_range']
df.drop('price_range',axis=1, inplace= True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Initialize and fit the KNN model
best_k = None
best_p = None
best_weights = None
best_accuracy = 0.0
k_values = range(1,11)
p_values = [1,2]
weight = ['uniform', 'distance']
for k in k_values:
    for p in p_values:
        for weights in weight:
            knn = KNeighborsClassifier(n_neighbors=k, p= p, weights = weights)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)*100
            print(f'Accuracy for k={k} with p{p} and weight {weights}: {accuracy}')
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_k = k
                best_p = p
                best_weights = weights
print(f"\nBest k value: {best_k}")
print(f"\nBest p value: {best_p}")
print(f"\nBest weights values: {best_weights}")

# Create the final model with the best k
final_model = KNeighborsClassifier(n_neighbors= best_k, p= best_p, weights = best_weights)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Model: ", accuracy*100)
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("\nPrecision Score:")
precision = precision_score(y_test, y_pred, average ='macro')
print(precision)
print("\nRecall Score:")
recall = recall_score(y_test, y_pred, average = 'macro')
print(recall)
print("\nF1 Score:")
f1score = f1_score(y_test, y_pred, average = 'macro')
print(f1score)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, make_scorer, recall_score, f1_score

df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/Mobile.csv')
X = df.drop('price_range',axis=1)
y = df['price_range']
df.drop('price_range',axis=1, inplace= True)

print('Using Accuracy Score: ')
cv_folds = 10
cv_scr=cross_val_score(knn, X, y, cv=cv_folds)
print('Cross Val Score: ', cv_scr)
print('Mean: ', np.mean(cv_scr))
print('Standard Deviation: ', np.std(cv_scr))

print('Using Precision Score: ')
precision_scorer = make_scorer(precision_score, average = 'macro')
cv_folds = 10
cv_scr=cross_val_score(knn, X, y, cv=cv_folds, scoring = precision_scorer)
print('Cross Val Score: ', cv_scr)
print('Mean: ', np.mean(cv_scr))
print('Standard Deviation: ', np.std(cv_scr))

print('Using Recall Score: ')
recall_scorer = make_scorer(recall_score, average = 'macro')
cv_folds = 10
cv_scr=cross_val_score(knn, X, y, cv=cv_folds, scoring = recall_scorer)
print('Cross Val Score: ', cv_scr)
print('Mean: ', np.mean(cv_scr))
print('Standard Deviation: ', np.std(cv_scr))

print('Using F1 Score: ')
f1_scorer = make_scorer(f1_score, average = 'macro')
cv_folds = 10
cv_scr=cross_val_score(knn, X, y, cv=cv_folds, scoring = f1_scorer)
print('Cross Val Score: ', cv_scr)
print('Mean: ', np.mean(cv_scr))
print('Standard Deviation: ', np.std(cv_scr))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/Mobile.csv')
X = df.drop('price_range',axis=1)
y = df['price_range']
df.drop('price_range',axis=1, inplace= True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
knn = KNeighborsClassifier()

k_range = list(range(1, 11))
weight = ['uniform', 'distance']
ps=[1,2]
param_grid = dict(n_neighbors=k_range,weights=weight,p=ps)
grid = GridSearchCV(knn, param_grid, cv=10, scoring= 'accuracy')
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train)
print(grid_search.best_params_) 
best_k=grid_search.best_estimator_.get_params()['n_neighbors'];
best_w=grid_search.best_estimator_.get_params()['weights'];
best_p=grid_search.best_estimator_.get_params()['p'];
print('Best K neighbors:', best_k)
print('Best weight:', best_w)
print('Beat P value:', best_p)

knn = KNeighborsClassifier(n_neighbors=best_k,weights=best_w,p=best_p);
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test) 
accuracy= accuracy_score(y_test,y_pred)*100
print("Accuracy of the Model: ", accuracy)
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("\nPrecision Score:")
precision = precision_score(y_test, y_pred, average ='weighted')
print(precision)
print("\nRecall Score:")
recall = recall_score(y_test, y_pred, average = 'weighted')
print(recall)
print("\nF1 Score:")
f1score = f1_score(y_test, y_pred, average = 'weighted')
print(f1score)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/Mobile.csv')
X = df.drop('price_range', axis=1)
y = df['price_range']
df.drop('price_range', axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe = Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

# Define the parameter grid including the number of neighbors, weights, and p
param_grid = {
    'knn__n_neighbors': list(range(1, 11)),
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]
}

grid = GridSearchCV(pipe, param_grid, cv=10, scoring='accuracy')

# fitting the model for grid search
grid_search = grid.fit(X_train, y_train)
print(grid_search.best_params_)
best_k=grid_search.best_estimator_.get_params()['knn__n_neighbors'];
best_w=grid_search.best_estimator_.get_params()['knn__weights'];
best_p=grid_search.best_estimator_.get_params()['knn__p'];
print('Best K neighbors:', best_k)
print('Best weight:', best_w)
print('Beat P value:', best_p)

knn = KNeighborsClassifier(n_neighbors=best_k,weights=best_w,p=best_p);
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test) 
accuracy= accuracy_score(y_test,y_pred)*100
print("Accuracy of the Model: ", accuracy)
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("\nPrecision Score:")
precision = precision_score(y_test, y_pred, average ='weighted')
print(precision)
print("\nRecall Score:")
recall = recall_score(y_test, y_pred, average = 'weighted')
print(recall)
print("\nF1 Score:")
f1score = f1_score(y_test, y_pred, average = 'weighted')
print(f1score)

In [20]:
# Regression Problem
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('C:/Users/ashis/OneDrive/Desktop/Machine Learning/insurance.csv')
print(df)
df.info()
print('Shape: ', df.shape)
print('Null Values Count: ')
print(df.isnull().sum())
print(df.duplicated().sum())
df.drop_duplicates(keep='first', inplace = True)
print(df.duplicated().sum())
print(df.shape)

df[['sex', 'smoker', 'region']] = df[['sex', 'smoker', 'region']].astype('category')
print(df.dtypes)

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
label.fit(df.sex.drop_duplicates())
df.sex = label.transform(df.sex)
label.fit(df.smoker.drop_duplicates())
df.smoker = label.transform(df.smoker)
label.fit(df.region.drop_duplicates())
df.region = label.transform(df.region)
print(df.dtypes)
print(df)

X = df.drop('charges', axis=1)
y = df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe = Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsRegressor())])

# Define the parameter grid including the number of neighbors, weights, and p
param_grid = {
    'knn__n_neighbors': list(range(1, 11)),
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]
}

grid = GridSearchCV(pipe, param_grid, cv=10, scoring='neg_mean_squared_error')

# fitting the model for grid search
grid_search = grid.fit(X_train, y_train)
print('Best Hyperparameters: ', grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred= best_model.predict(X_test) 

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error: ", rmse)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    -----------